In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from sklearn import linear_model

In [ ]:
from predict_sales import logger

In [ ]:
from predict_sales.data import Data, log_lm_features, linear_features

In [ ]:
Data.process_input("../input/store.csv", "../input/train.csv", "../input/test.csv")

In [ ]:
Data.save('train.pkl', 'test.pkl')

In [ ]:
from predict_sales import functions

In [ ]:
def check_dup_columns(df):
    return len(df.columns) == len(df.columns.unique())

In [ ]:
check_dup_columns(Data.test)

In [ ]:
import pickle
with open('train.pkl', 'rb') as pkl_file:
    train = pickle.load(pkl_file)
    logger.info('Loaded train data, shape {0}'.format(train.shape))
with open('test.pkl', 'rb') as pkl_file:
    test = pickle.load(pkl_file)
    logger.info('Loaded test data, shape {0}'.format(test.shape))

In [ ]:
train.shape

In [ ]:
train1 = functions.remove_outliers_lm(train)

In [ ]:
isinstance(A['Id'], pd.DataFrame)

In [ ]:
train1.shape

In [ ]:
len(list(linear_features)), len(list(log_lm_features))

In [ ]:
set(log_lm_features).difference(linear_features)

In [ ]:
test = Data.test.copy()

In [ ]:
check_dup_columns(test)

In [ ]:
train['Id'] = np.arange(1, train.shape[0] + 1)

In [ ]:
check_dup_columns(train)

In [ ]:
train = functions.select_features(train, log_lm_features)

In [ ]:
train.shape

In [ ]:
l = [(1,2),'ab']

In [ ]:
def mygen():
    for i in range(10):
        yield Predictions(i, 3*i, 5*i)

In [ ]:
Predictions._fields

In [ ]:
list(mygen())

In [ ]:
Predictions(*zip(*mygen()))

In [ ]:
functions.predict_lm(train, train, save_fit=True)

In [ ]:
lm = linear_model.LinearRegression()

In [ ]:
test.shape

In [ ]:
pd.isnull(train).any()

In [ ]:
fit = lm.fit(train.drop(['Date', 'Sales'], axis=1), train['Sales'])

In [ ]:
pred = lm.predict(train.drop(['Date', 'Sales'], axis=1))

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
pd.Series(pred).describe()

In [ ]:
train['Sales'].describe()

In [ ]:
A = train.iloc[:10,:2]

In [ ]:
abs(A['Sales']-5000), A['Sales']-5000

In [ ]:
pd.DataFrame(mygen())

In [ ]:
pd.concat(map(lambda x: pd.DataFrame([x]), mygen()))

In [ ]:
plt.scatter(y=pred, x=train['Sales'])
plt.xlim([0,25000])

In [ ]:
functions.predict_lm_per_store()

In [ ]:
del plt

In [ ]:
train_gb = train.groupby('Store')

In [ ]:
train_gb.get_group(1).head()

In [ ]:
from collections import namedtuple

In [ ]:
Predictions = namedtuple('Predictions', 'predicted, fit, store')

In [ ]:
p = Predictions(predicted=0, fit=1, store=None)
p

In [ ]:
p._replace(store=7)